In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
import pandas as pd
import numpy as np
import PyPDF2
import re

from io import StringIO
from html.parser import HTMLParser

load additional packages for NLTK

In [ ]:
nltk.download('punkt')

Rabota.ru jobs dataset was chosen for training - it's the largest one and the most diverse one probably (jobs are distributed all across Russia and IT is probably less prevalent here than on hh.ru).

In [ ]:
df = pd.read_csv('data/vacancy.csv', sep='|')
df.head()

Preprocessing - remove HTML tags, lowercase everything, remove punctuation

In [ ]:
# HTML stripping (https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python)
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [ ]:
df[['a_req', 'p_req', 'p_res']] = df[['additional_requirements', 'position_requirements' , 'position_responsibilities']].astype(str)
df['total_req'] = df['a_req'] + ' ' + df['p_req'] + ' ' + df['p_res']
df['total_req'] = df['total_req'].str.replace('nan', '').str.strip(' ')
df['total_req'] = df['total_req'].apply(lambda x: re.sub(r'[^\w\s]', '', strip_tags(x.lower())))  # remove HTML tags, lower case, remove punctuation
df.head()

In [ ]:
data = df['total_req'].values.tolist()
tagged_data = [TaggedDocument(words = nltk.tokenize.word_tokenize(_d), tags = [str(i)]) for i, _d in enumerate(data)]

init model

In [ ]:
model = Doc2Vec(
    vector_size = 50,
    min_count = 10,
    epochs = 50
)

build vocabulary

In [ ]:
model.build_vocab(tagged_data)
len(model.wv.key_to_index)  # vocab size

train model

In [ ]:
model.train(tagged_data,
total_examples = model.corpus_count,
epochs = model.epochs)
model.save('model/doc2vec_v2.model')